<a href="https://colab.research.google.com/github/TuragDev/ML-Python-Codes/blob/main/Crystal_system_prediction_by_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.3 MB/s eta 0:00:00


In [2]:
pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.3 MB/s eta 0:00:00


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier  # Replace with the appropriate ANN class
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import joblib
from sklearn.model_selection import train_test_split
import numpy as np

import pandas as pd

from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split

import numpy as np

import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import sys
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV

In [4]:
from google.colab import files
uploaded= files.upload()

Saving Orbital_mp_dataset.csv to Orbital_mp_dataset.csv
Saving sine_mp_data.csv to sine_mp_data.csv
Saving Xrd_mp_dataset.csv to Xrd_mp_dataset.csv


In [5]:
df= pd.read_csv('Xrd_mp_dataset.csv')

In [6]:
X=df.iloc[:,14:]
Y=df['crystal_system']

In [7]:
X.shape

(745, 128)

In [ ]:
from sklearn.preprocessing import StandardScaler
# create a scaler object
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X)

# apply the scaler to both the training and test data
X_normalized = scaler.transform(X)


label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_encoded = label_encoder.fit_transform(Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=108)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from kerastuner import HyperParameters

hp = HyperParameters()


def build_multiclass_model(hp):
    # Input layer
    inputs = tf.keras.Input(shape=(128,))

    # Apply a dense layer with ReLU activation
    x = tf.keras.layers.Dense(
        units=hp.Int('dense_units', min_value=32, max_value=1024, step=32, default=32),
        activation='relu',
    )(inputs)

    # Add more dense layers based on hp.Int('dense_layers', ...)
    for i in range(hp.Int('dense_layers', 1, 8, default=1)):
        x = tf.keras.layers.Dense(
            units=hp.Int('dense_units_'+str(i), min_value=32, max_value=1024, step=32, default=32),
            activation='relu',
        )(x)
        x = tf.keras.layers.Dropout(hp.Float('dropout_dense_'+str(i), 0.1, 0.8, step=0.1, default=0.3))(x)

    # Output layer for multiclass classification
    num_classes = 7  # Replace with the actual number of classes in your dataset
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

    # Create the model
    model = tf.keras.Model(inputs, outputs)

    # Compile the model for multiclass classification
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-01, 1e-2, 1e-3, 1e-4])
        ),
        loss='categorical_crossentropy',  # Use categorical cross-entropy for multiclass
        metrics=['accuracy']  # Add more metrics as needed
    )

    return model


In [ ]:
model = build_multiclass_model(hp)

In [ ]:
import numpy as np
from sklearn.model_selection import RepeatedKFold, cross_val_score
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# Define a function to create the Keras model
def create_model(hp):
    model = build_multiclass_model(hp)
    return model

# Create a KerasClassifier wrapper for your Keras model
keras_wrapper = KerasClassifier(build_fn=create_model, hp=hp, epochs=10, batch_size=32, verbose=0)

# Use 5-fold cross-validation repeated 10 times (adjust as needed)
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)

# Perform cross-validation and get the scores
scores = cross_val_score(keras_wrapper, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

# Calculate the mean accuracy across all folds and repetitions
mean_accuracy = np.mean(scores)
print("Mean Accuracy:", mean_accuracy)


<ipython-input-29-545b86fc1d31>:11: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_wrapper = KerasClassifier(build_fn=create_model, hp=hp, epochs=10, batch_size=32, verbose=0)


Mean Accuracy: 0.6216386554621849


In [ ]:
keras_wrapper.fit(X_train,y_train, batch_size =20 )
pred = keras_wrapper.predict(X_train)
training_accuracy= accuracy_score(y_train,pred)
print("Accuracy on Training data: ",training_accuracy)


19/19 [==============================] - 0s 4ms/step
Accuracy on Training data:  0.6241610738255033


In [ ]:
pred = keras_wrapper.predict(X_test)
test_accuracy= accuracy_score(y_test,pred)
print("Accuracy on Test data: ",test_accuracy)


5/5 [==============================] - 0s 3ms/step
Accuracy on Test data:  0.5704697986577181


In [ ]:
df= pd.read_csv('sine_mp_data.csv')
X=df.iloc[:,14:]
Y=df['crystal_system']

In [ ]:
X.shape

(745, 152)

In [ ]:
from sklearn.preprocessing import StandardScaler
# create a scaler object
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X)

# apply the scaler to both the training and test data
X_normalized = scaler.transform(X)


label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_encoded = label_encoder.fit_transform(Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=108)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from kerastuner import HyperParameters

hp = HyperParameters()


def build_multiclass_model(hp):
    # Input layer
    inputs = tf.keras.Input(shape=(152,))

    # Apply a dense layer with ReLU activation
    x = tf.keras.layers.Dense(
        units=hp.Int('dense_units', min_value=32, max_value=1024, step=32, default=32),
        activation='relu',
    )(inputs)

    # Add more dense layers based on hp.Int('dense_layers', ...)
    for i in range(hp.Int('dense_layers', 1, 8, default=1)):
        x = tf.keras.layers.Dense(
            units=hp.Int('dense_units_'+str(i), min_value=32, max_value=1024, step=32, default=32),
            activation='relu',
        )(x)
        x = tf.keras.layers.Dropout(hp.Float('dropout_dense_'+str(i), 0.1, 0.8, step=0.1, default=0.3))(x)

    # Output layer for multiclass classification
    num_classes = 7  # Replace with the actual number of classes in your dataset
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

    # Create the model
    model = tf.keras.Model(inputs, outputs)

    # Compile the model for multiclass classification
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-01, 1e-2, 1e-3, 1e-4])
        ),
        loss='categorical_crossentropy',  # Use categorical cross-entropy for multiclass
        metrics=['accuracy']  # Add more metrics as needed
    )

    return model


In [ ]:
model = build_multiclass_model(hp)

In [ ]:
import numpy as np
from sklearn.model_selection import RepeatedKFold, cross_val_score
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# Define a function to create the Keras model
def create_model(hp):
    model = build_multiclass_model(hp)
    return model

# Create a KerasClassifier wrapper for your Keras model
keras_wrapper = KerasClassifier(build_fn=create_model, hp=hp, epochs=10, batch_size=32, verbose=0)

# Use 5-fold cross-validation repeated 10 times (adjust as needed)
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)

# Perform cross-validation and get the scores
scores = cross_val_score(keras_wrapper, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

# Calculate the mean accuracy across all folds and repetitions
mean_accuracy = np.mean(scores)
print("Mean Accuracy:", mean_accuracy)


<ipython-input-40-545b86fc1d31>:11: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_wrapper = KerasClassifier(build_fn=create_model, hp=hp, epochs=10, batch_size=32, verbose=0)


Mean Accuracy: 0.6291974789915966


In [ ]:
keras_wrapper.fit(X_train,y_train, batch_size =20 )
pred = keras_wrapper.predict(X_train)
training_accuracy= accuracy_score(y_train,pred)
print("Accuracy on Training data: ",training_accuracy)


19/19 [==============================] - 0s 2ms/step
Accuracy on Training data:  0.6291946308724832


In [ ]:
pred = keras_wrapper.predict(X_test)
test_accuracy= accuracy_score(y_test,pred)
print("Accuracy on Test data: ",test_accuracy)


5/5 [==============================] - 0s 4ms/step
Accuracy on Test data:  0.5503355704697986


In [8]:
df= pd.read_csv('Orbital_mp_dataset.csv')


X=df.iloc[:,14:]
Y=df['crystal_system']

In [9]:
X.shape

(744, 1024)

In [10]:
from sklearn.preprocessing import StandardScaler
# create a scaler object
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X)

# apply the scaler to both the training and test data
X_normalized = scaler.transform(X)


label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_encoded = label_encoder.fit_transform(Y)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=108)

In [12]:
import tensorflow as tf
from tensorflow import keras
from kerastuner import HyperParameters

hp = HyperParameters()


def build_multiclass_model(hp):
    # Input layer
    inputs = tf.keras.Input(shape=(1024,))

    # Apply a dense layer with ReLU activation
    x = tf.keras.layers.Dense(
        units=hp.Int('dense_units', min_value=32, max_value=1024, step=32, default=32),
        activation='relu',
    )(inputs)

    # Add more dense layers based on hp.Int('dense_layers', ...)
    for i in range(hp.Int('dense_layers', 1, 8, default=1)):
        x = tf.keras.layers.Dense(
            units=hp.Int('dense_units_'+str(i), min_value=32, max_value=1024, step=32, default=32),
            activation='relu',
        )(x)
        x = tf.keras.layers.Dropout(hp.Float('dropout_dense_'+str(i), 0.1, 0.8, step=0.1, default=0.3))(x)

    # Output layer for multiclass classification
    num_classes = 7  # Replace with the actual number of classes in your dataset
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

    # Create the model
    model = tf.keras.Model(inputs, outputs)

    # Compile the model for multiclass classification
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-01, 1e-2, 1e-3, 1e-4])
        ),
        loss='categorical_crossentropy',  # Use categorical cross-entropy for multiclass
        metrics=['accuracy']  # Add more metrics as needed
    )

    return model


<ipython-input-12-f95a3c2c24f8>:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperParameters


In [13]:
model = build_multiclass_model(hp)

In [14]:
import numpy as np
from sklearn.model_selection import RepeatedKFold, cross_val_score
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# Define a function to create the Keras model
def create_model(hp):
    model = build_multiclass_model(hp)
    return model

# Create a KerasClassifier wrapper for your Keras model
keras_wrapper = KerasClassifier(build_fn=create_model, hp=hp, epochs=10, batch_size=32, verbose=0)

# Use 5-fold cross-validation repeated 10 times (adjust as needed)
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)

# Perform cross-validation and get the scores
scores = cross_val_score(keras_wrapper, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

# Calculate the mean accuracy across all folds and repetitions
mean_accuracy = np.mean(scores)
print("Mean Accuracy:", mean_accuracy)


<ipython-input-14-545b86fc1d31>:11: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_wrapper = KerasClassifier(build_fn=create_model, hp=hp, epochs=10, batch_size=32, verbose=0)


Mean Accuracy: 0.6124369747899161


In [15]:
keras_wrapper.fit(X_train,y_train, batch_size =20 )
pred = keras_wrapper.predict(X_train)
training_accuracy= accuracy_score(y_train,pred)
print("Accuracy on Training data: ",training_accuracy)


19/19 [==============================] - 0s 2ms/step
Accuracy on Training data:  0.6134453781512605


In [16]:
pred = keras_wrapper.predict(X_test)
test_accuracy= accuracy_score(y_test,pred)
print("Accuracy on Test data: ",test_accuracy)


5/5 [==============================] - 0s 4ms/step
Accuracy on Test data:  0.6174496644295302
